In [18]:
class Descriptor:
    attribute_name: str
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name 

    def __set__(self, instance, value):
        '''
            self is the descriptor instance (this class)
            instance is the MANAGED insance 
            Descriptors should store values in managed instances

            E.g of what NOT to do, show what happens if I do
                self.__dict__[self.attribute_name] = value
            this modifies the class attribute for ALL Descriptor classes!
        '''
        if value < 0:
            raise ValueError
        instance.__dict__[self.attribute_name] = value


In [19]:
d =Descriptor('some_name')

In [20]:
class ManagedClass:
    attr = Descriptor('attr')

    def __init__(self, attr):
        self.attr = attr


In [26]:
m = ManagedClass(-10)

ValueError: 

In [27]:
# Doing it without repeating names

class Quantity:
    __numinstance = 0 # class attribute across ALL instances

    def __init__(self, ):
        cls = self.__class__ # cls refers to the Quantity class
        prefix = cls.__name__
        index = cls.__numinstance

        self.attr_name = f'_{prefix}#{index}' # unique!
        cls.__numinstance += 1 

    def __get__(self, instance, owner):
        return getattr(instance, self.attr_name) # need to implement this because name of managed attribute is NOT the same as the attr_name
        # getattr used here bc names are different, will not trigger infinite loop

    def __set__(self, instance, value):
        setattr(instance, self.attr_name, value)


NameError: name 'm' is not defined

In [ ]:
# have a parent Quantity

# create a Validated abstract class
import abc

class Validated(abc.ABC, Quantity):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value) # THIS performans the actual storage, in this case the set method in Quantity

    @abc.abstractmethod
    def validate(self, instance, value):
        '''Allows subclasses to implement their own validation'''

class ValidationOne(Validated):
    '''no numbers outsize 0 to 1'''
    def validate(self, instance, value):
        if value < 0 or value > 1:
            raise ValueError
        return value

class ValidationTwo(Validated):
    '''No non-integers'''
    def validate(self, instance, value):
        if not isinstance(value, int):
            raise ValueError

# Important thing is that USERS do NOT need to know the internals, they just get validated (idiot-proof) attributes with useful error messages
